In [1]:
import ROOT
import numpy as np
from itertools import combinations_with_replacement
from ast import literal_eval

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8c7b7d0


In [2]:
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-reco_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-reco_jm_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-clean_reco_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-clean_reco_jm_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-cheat_clean_reco_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-cheat_clean_reco_jm_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-mc_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-nomb_mc_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-nomb_mc_rlep_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-av_mc_oo.txt"
input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-av_nomb_mc_oo.txt"
# input_path = "fit-configs/signal-only/fit-inputs-0.0-0.0-av_nomb_mc_rlep_oo.txt"

In [3]:
with open(input_path) as infile:
    lines = infile.readlines()
n_per_ab = literal_eval(lines[1])
nominal_means = literal_eval(lines[2])
slopes = literal_eval(lines[3])
Cov_O = literal_eval(lines[4])
print(lines)

['#evt/ab_inv, means, slopes, cov\n', '2047262.771679877\n', '[-0.10906062831297444, -0.05495127051162606, 0.08114173834790378]\n', '[[-6.700568339685052, -1.4004801624952383, 4.313173786875001], [-2.3731033421744256, -4.8713117836717785, 1.4806120387846007], [-5.808097233476862, -0.935398530140426, 11.348726891618908]]\n', '[[0.7049211901002926, 0.13872256861291385, -0.4512026311375545], [0.13872256861291385, 0.2569047321153842, -0.07231313210080946], [-0.4512026311375545, -0.07231313210080946, 0.8947573732880347]]\n']


In [4]:
n_obs = len(nominal_means)
run_ab = 2
# run_ab = 11.2

In [5]:
coupling_names = ["g1z", "ka", "lz"]

In [6]:
Cov_mean_O = np.asarray(Cov_O) / (n_per_ab * run_ab)

In [7]:
# expected precision:
np.sqrt(np.diag(np.linalg.inv(np.asarray(Cov_O))) / (n_per_ab * run_ab))

array([0.00074866, 0.00103236, 0.00063548])

In [8]:
couplings = []
for i in range(3):
    cpl = ROOT.RooRealVar(coupling_names[i], coupling_names[i], 0., -0.5, 0.5)
    couplings.append(cpl)

obs = []
mus_exp = []
all_vars = []
rel_changes = []
for i, m in enumerate(nominal_means):
    sigma = np.sqrt(Cov_mean_O[i,i])
    o = ROOT.RooRealVar(f"O_{i+1}", f"O_{i+1}", m, m-5*sigma, m+5*sigma)
    obs.append(o)

    vars = []
    for j in range(3):
        # var = ROOT.RooProduct(f"var_{i}_{j}", f"var_{i}_{j}", couplings[j], ROOT.RooFit.RooConst(slopes[j][i]))
        var = ROOT.RooProduct(f"var_{i}_{j}", f"var_{i}_{j}", couplings[j], ROOT.RooFit.RooConst(slopes[i][j]))
        vars.append(var)
    all_vars += vars
    nominal_exp = ROOT.RooFit.RooConst(m)
    # mu_exp = ROOT.RooAddition(f"mu_exp_{i}", f"mu_exp_{i}", [nominal_exp] + vars)
    rel_change = ROOT.RooAddition(f"rel_change_{i}", f"rel_change_{i}", [ROOT.RooFit.RooConst(1.0)] + vars)
    rel_changes.append(rel_change)
    mu_exp = ROOT.RooProduct(f"mu_exp_{i}", f"mu_exp_{i}", rel_change, nominal_exp)
    mus_exp.append(mu_exp)

print(all_vars)

Cov_mean_O_root = ROOT.TMatrixDSym(n_obs)
for i, j in combinations_with_replacement(range(3), r=2):
    Cov_mean_O_root[i][j] = Cov_mean_O[i, j]
    Cov_mean_O_root[j][i] = Cov_mean_O[i, j]

model = ROOT.RooMultiVarGaussian("model", "model", obs, mus_exp, Cov_mean_O_root)

[<cppyy.gbl.RooProduct object at 0xbd1cfb0>, <cppyy.gbl.RooProduct object at 0xbd197e0>, <cppyy.gbl.RooProduct object at 0xbd1bcb0>, <cppyy.gbl.RooProduct object at 0xc0234d0>, <cppyy.gbl.RooProduct object at 0xc0a21e0>, <cppyy.gbl.RooProduct object at 0xc0b2bd0>, <cppyy.gbl.RooProduct object at 0xbfd4dd0>, <cppyy.gbl.RooProduct object at 0xc015530>, <cppyy.gbl.RooProduct object at 0xc0aec90>]


In [9]:
model.Print("t")

0xc358c20 RooMultiVarGaussian::model = 1 [Auto,Dirty] 
  0x9712870/V- RooRealVar::O_1 = -0.109061
  0x53eb6e0/V- RooRealVar::O_2 = -0.0549513
  0xbed1830/V- RooRealVar::O_3 = 0.0811417
  0xc07cb20/V- RooProduct::mu_exp_0 = -0.109061 [Auto,Clean] 
    0xb6b8ce0/V- RooAddition::rel_change_0 = 1 [Auto,Clean] 
      0xbdb3490/V- RooConstVar::1 = 1
      0xbd1cfb0/V- RooProduct::var_0_0 = 0 [Auto,Clean] 
        0xb9e4000/V- RooRealVar::g1z = 0
        0xbcc6330/V- RooConstVar::-6.70057 = -6.70057
      0xbd197e0/V- RooProduct::var_0_1 = 0 [Auto,Clean] 
        0xb9ec7e0/V- RooRealVar::ka = 0
        0xbc06c10/V- RooConstVar::-1.40048 = -1.40048
      0xbd1bcb0/V- RooProduct::var_0_2 = 0 [Auto,Clean] 
        0xb68f250/V- RooRealVar::lz = 0
        0xbc07740/V- RooConstVar::4.31317 = 4.31317
    0xbdb3f00/V- RooConstVar::-0.109061 = -0.109061
  0x53fe9f0/V- RooProduct::mu_exp_1 = -0.0549513 [Auto,Clean] 
    0xbb89d80/V- RooAddition::rel_change_1 = 1 [Auto,Clean] 
      0xbdb3490/V- RooCons

In [10]:
w = ROOT.RooWorkspace("w", "workspace")
# w.Import(model)
modelconfig = ROOT.RooStats.ModelConfig("modelconfig", w)
modelconfig.SetPdf(model)
modelconfig.SetParametersOfInterest(couplings)
modelconfig.SetObservables(obs)
# w.Print()
w.Import(modelconfig)
w.writeToFile("gen-lvl-standalone-ws.root")
w

In [11]:
ds = ROOT.RooStats.AsymptoticCalculator.GenerateAsimovData(model, obs)
ds.Print("v")

DataStore CountingAsimovData0 (CountingAsimovData0)
  Contains 1 entries
  Observables: 
    1)  O_1 = -0.109061  L(-0.111135 - -0.106986)  "O_1"
    2)  O_2 = -0.0549513  L(-0.0562037 - -0.0536988)  "O_2"
    3)  O_3 = 0.0811417  L(0.0788044 - 0.0834791)  "O_3"


In [12]:
nll = model.createNLL(ds)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_over_model_Int[O_1,O_2,O_3]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 7.93434 ms


In [13]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[O_1,O_2,O_3]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level


In [14]:
nll_minimizer.migrad()
res = nll_minimizer.lastMinuitFit()
# res.Print()
res.correlationMatrix().Print()
# print(f"g1z: {res.error(0)}, ka: {res.error(1)}, lz: {res.error(2)}")

Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = -21.2429106927740179
Edm   = 2.41415731647022122e-28
Nfcn  = 40
g1z	  = 0	 +/-  0.000723095	(limited)
ka	  = 0	 +/-  0.000985545	(limited)
lz	  = 0	 +/-  0.000622696	(limited)

3x3 matrix is as follows

     |      0    |      1    |      2    |
--------------------------------------------
   0 |          1      -0.271      0.5628 
   1 |     -0.271           1    -0.02036 
   2 |     0.5628    -0.02036           1 



Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 307.989 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -21.24291069 Edm =   1.604106047e-28 NCalls =     13
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : -21.24291069
  Edm           : 1.604106047e-28
  Internal parameters:	[                0                0                0]	
  Internal gradient  :	[                0                0 -1.764377416e-11]	
  Internal covariance matrix:
[[  2.5763988e-06              0              0]
 [              0  7.0014146e-06              0]
 [              0              0  2.0611523e-06]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1500
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =      -21.24291069 Edm =   1.604106047e-28 NCalls =     13
Warning in <Minuit2>: VariableMetricBuilder No improveme

In [15]:
# errors seem to be very symmetric, no need for this
# nll_minimizer.minos(couplings)

In [16]:
print(couplings[0].getError())
print(couplings[1].getError())
print(couplings[2].getError())

0.0007230948662635561
0.000985544728217144
0.0006226961540355423


In [17]:
pll0 = nll.createProfile({couplings[0]})
pll1 = nll.createProfile({couplings[1]})
pll2 = nll.createProfile({couplings[2]})

In [18]:
frame0 = couplings[0].frame(Range=(-0.004, 0.004), Title=f";{coupling_names[0]};#Delta NLL")
frame1 = couplings[1].frame(Range=(-0.004, 0.004), Title=f";{coupling_names[1]};#Delta NLL")
frame2 = couplings[2].frame(Range=(-0.004, 0.004), Title=f";{coupling_names[2]};#Delta NLL")
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

In [19]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[O_1,O_2,O_3]_CountingAsimovData0) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) minimum found at (g1z=0)
..........................................................................................................................................................................................................
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[ka]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[O_1,O_2,O_3]_CountingAsimovData0) Summation contains a RooNLLVar

In [20]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(2)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")
# c0.SaveAs("plots/fit/ll_pll_g1z.pdf")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(2)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")
# c1.SaveAs("plots/fit/ll_pll_ka.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(2)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")
# c2.SaveAs("plots/fit/ll_pll_la.pdf")